## This notebook is made for comparison between dataset provided by loading the whole one into notebook vs the using a generator

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13611526205076128243]

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [9]:
width, height = 64, 64
train_data_dir = '../kostki/zdjecia_gen/train'
test_data_dir = '../kostki/zdjecia_gen/test'
nb_train_samples = 20160
nb_test_samples = 5040
epochs = 25
batch_size = 256

input_shape = (width, height, 3)

In [11]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
# summarize model and apply checkpoints
print(model.summary())
filepath = 'generator_comparison-{epoch:02d}-{loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
activation_5 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18496     
__________

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(width, height),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(width, height),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

Found 20160 images belonging to 6 classes.
Found 5040 images belonging to 6 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks_list,
    shuffle=True,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

Epoch 1/25
78/78 [==============================] - 73s 937ms/step - loss: 1.7756 - acc: 0.1889 - val_loss: 1.6726 - val_acc: 0.2975

Epoch 00001: loss improved from inf to 1.77552, saving model to generator_comparison-01-1.7755.h5
Epoch 2/25
78/78 [==============================] - 68s 873ms/step - loss: 1.6072 - acc: 0.3080 - val_loss: 1.4619 - val_acc: 0.3886

Epoch 00002: loss improved from 1.77552 to 1.60715, saving model to generator_comparison-02-1.6071.h5
Epoch 3/25
78/78 [==============================] - 73s 933ms/step - loss: 1.3376 - acc: 0.4231 - val_loss: 1.1094 - val_acc: 0.5512

Epoch 00003: loss improved from 1.60715 to 1.33818, saving model to generator_comparison-03-1.3382.h5
Epoch 4/25
78/78 [==============================] - 72s 928ms/step - loss: 1.1139 - acc: 0.5231 - val_loss: 0.9290 - val_acc: 0.6369

Epoch 00004: loss improved from 1.33818 to 1.11395, saving model to generator_comparison-04-1.1140.h5
Epoch 5/25
78/78 [==============================] - 72s 921m

In [ ]:
print(history.history.keys())
print("Accuracy: %.2f%%" % (score[1]*100))
print('Test loss:', score[0])
print('Test accuracy', score[1])

In [ ]:
#summarize for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#summarize for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model_json = model.to_json()
with open('firstgenerator_model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('firstgenerator_weights.h5')
print('Saved model weights')